# Backend Verification for the iHP Open MPW Shuttles

[D. Mitch Bailey](https://www.linkedin.com/in/mitch-bailey-cvc/), [ShuhariSystem](https://www.shuharisystem.com/)

# Overview
This notebook performs backend LVS (with [klayout](https://github.com/KLayout/klayout) or [magic](https://github.com/RTimothyEdwards/magic)/[netgen](https://github.com/RTimothyEdwards/netgen)), soft-connection checks (with magic/netgen) and/or reliability verfication (with [CVC-RV](https://github.com/d-m-bailey/cvc)) with open source EDA software and systems.

# iHP Open-MPW Data
The 17 submissions to iHp open mpw shuttle for April 2025 listed below are located [here](https://github.com/IHP-GmbH/TO_Apr2025).

* 160GHz_LNA
* 40_GHZ_LOW_NOISE_TIA
* 6502-cpu
* 97_GHZ_LINEAR_TIA
* DC_to_130_GHz_TIA
* GPS_LNA
* Greyhound
* Mixer5GHz
* PA_180GHz
* TTIHP0p2
* TTIHP25a
* VCO_130nm_LSI
* active_L_VCOs
* ascon
* bandgap_ref_cmos
* elemrv-n
* i2c-gpio-expander

Create a file with the initial environment variables.


In [1]:
%%writefile /content/env
export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi


Writing /content/env


# Program installation.
Only needs to be executed once.

This step sets up the pdk and installs magic, klayout, netgen and cvc_rv.

Runtime: 3-4 minutes.

In [2]:
%%shell
cd
cat /content/env
source /content/env

lsb_release -a 2>/dev/null

if [[ ! -d $PDK_ROOT ]]; then
  echo "==> Installing ciel..."
  pip install ciel
fi
rm -rf $PDK_ROOT
ciel enable --pdk $PDK $PDK_COMMIT
# patch for mimcap and antenna error in netgen setup file from pdk
# patch to remove permute for npn devices.
# patch to remove parallel combination of bipolar devices. parallel devices with matching parameters will be combined during magic extraction.
# patch to change cap_rfcmim to rfcmim.
# patch to remove wfeed parameter from source capacitors because magic does not extract it.
sed -i.bak -e '/lsearch .cells2/,/circuit2/s/circuit1 .dev/circuit2 $dev/' \
  -e '/npn13/,/circuit2.*delete/s/.*permute/#&/' \
  -e '/npn13/,/circuit2.*delete/s/.*parallel/#&/' \
  -e 's/cap_rfcmim/rfcmim/' \
  -e '/rfcmim/,/circuit2.*delete/s/.*circuit2.*delete.*/& wfeed/' $PDK_ROOT/$PDK/libs.tech/netgen/ihp-sg13g2_setup.tcl

# allow met7 text to be recognized on pads
sed -i.bak -e '/labels PADID/a\
labels MET7PIN port\
labels MET7TXT text
' \
  -e '/pad/!s/\*m7,m7fill,pillar,solder/&,pad/g' $PDK_ROOT/$PDK/libs.tech/magic/ihp-sg13g2.tech

# ignore rule that creates separate taps for labeled wells
# ignore gaurdring when computing ptap_holes
sed -i.bak -e '/^well_patt/s/"/"-/' -e '/^sub_patt/s/"/"-/' \
  -e 's/ptap_holes = .*/ptap_holes = ptap.not(edgeseal_drw).holes/' $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/general_derivations.lvs

# fix rfmim_sub
sed -i.bak -e 's/rfmim_sub =.*/rfmim_sub = rfmim_area.sized(1.um)/' $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/cap_derivations.lvs

# fix cap reduction
mv $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/custom_mim_extractor.lvs $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/custom_mim_extractor.lvs.org
awk '\
/Adding extra param.*rfcmim/ { print "    self.combiner = MIMCAPNDeviceCombiner.new"; } \
/if name.downcase.include...rfcmim/ { comment += 1; } \
comment == 1 { $1 = "#" $1; } \
comment == 1 && $1 == "#end" { comment += 1; } \
 { print $0; }' \
  $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/custom_mim_extractor.lvs.org > \
  $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/custom_mim_extractor.lvs

echo "==> Using pdk $PDK commit $PDK_COMMIT (patched)
"

# if [[ ! -d extra_be_checks ]]; then
  echo "==> Downloading extra_be_checks $EXTRA_CHECK_COMMIT"
  rm -rf extra_be_checks
  git clone https://github.com/d-m-bailey/extra_be_checks.git -b $EXTRA_CHECK_COMMIT
  mv $PDK_ROOT/$PDK/libs.tech/magic/ihp-sg13g2.tech $PDK_ROOT/$PDK/libs.tech/magic/ihp-sg13g2.tech.org
  cp extra_be_checks/tech/ihp-sg13g2/ihp-sg13g2.tech $PDK_ROOT/$PDK/libs.tech/magic/ihp-sg13g2.tech
  # Exchange npn13g2 l/w params
  sed -i.bak -e 's/w1=we l1=le/w1=le l1=we/' $PDK_ROOT/$PDK/libs.tech/magic/ihp-sg13g2.tech
# fi
echo "==> Using extra_be_checks commit $(cd extra_be_checks; git rev-parse HEAD)
"

if ! command -v netgen; then
  echo "==> Downloading and installing netgen $NETGEN_COMMIT"
  git clone https://github.com/RTimothyEdwards/netgen.git --depth=1 -b $NETGEN_COMMIT
  cd netgen
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using netgen version $(netgen -batch | awk '/Netgen/ {print $2}')
"

if ! command -v magic; then
  echo "==> Downloading and installing magic $MAGIC_COMMIT"
  git clone https://github.com/RTimothyEdwards/magic.git --depth=1 -b $MAGIC_COMMIT
  cd magic
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using magic version $(magic -dnull -noc --version)
"

if ! command -v klayout; then
  echo "==> Downloading and installing klayout $KLAYOUT_DEB for $KLAYOUT_DISTRO"
  wget -P /root https://www.klayout.org/downloads/$KLAYOUT_DISTRO/$KLAYOUT_DEB
  sudo apt install /root/$KLAYOUT_DEB
  pip install docopt
  pip install klayout
  cd
fi
echo "==> Using $(klayout -v)
"

if ! command -v cvc_rv; then
  echo "==> Downloading and installing cvc_rv $CVC_COMMIT"
  sudo apt install autopoint bison flex
  git clone https://github.com/d-m-bailey/cvc --depth=1 -b $CVC_COMMIT
  cd cvc
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL --disable-nls
  make
  make install
  cd
fi
echo "==> Using $(cvc_rv -v)
"

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy
==> Installing ciel...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 6.7 MB/s eta 0:00:00
Version ddb601a4a4473163e1ed6df416b885df18b4ac03 not found locally, attempting 
to download…
⠙ Unpacking common.tar.zst…
⠹ Unpacking sg13g2_io.tar.zst…
⠋ Unpacking sg13g2

# Clone the design repo and list the gds, spice and verilog files.

Run to refresh the design data.
The file list is also saved in /content/filelist.txt.

Runtime: 1-2 minutes.

In [3]:
%%shell
cat /content/env
source /content/env
rm -rf $DATA_ROOT $CHECK_ROOT
git clone https://github.com/d-m-bailey/ihp-mpw-be.git --depth=1 -b TO_Apr2025 $CHECK_ROOT
git clone https://github.com/IHP-GmbH/$MPW.git --depth=1 $DATA_ROOT
cd data
# uncompress gds, spice, cdl, and verilog files
find . \( -name "*.gds.gz" -o -name "*.spice.gz" -o -name "*.cdl.gz" -o -name "*.v.gz" \) -execdir gunzip -v {} \;
find . \( -name "*.gds.zip" -o -name "*.spice.zip" -o -name "*.cdl.zip" -o -name "*.v.zip" \) -execdir 7z x {} \;
# Create a file listing the size, project and relevant file names.
# Only list files that end with .gds (in gds subdirectories), .spice, .cdl, or .v.
tee /content/filelist.txt <<EOF
   Size Project                   File
------- ------------------------- ---------------------------------------------
EOF
ls -s $(find . \( -name "*.gds" -path "*/gds/*" \) -o -name "*.spice" -o -name "*.cdl" -o -name "*.v") |
  sed -e 's,\./,,' -e 's,/, ,' |
  awk '{printf "%7d %-25s %s\n", $1, $2, $3}' |
  tee -a /content/filelist.txt

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
Cloning into '/content/check_data'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 1), reused 21 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 130.62 KiB | 5.68 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Cloning into 

# ※ Run cells before this cell for program installation and general setup.<br>Then choose one setup from the 17 designs below.<br>Finally, run the checks and examine the results.

# 1. 160GHz_LNA

Checks will not work with GDS file in repo.
1. Copy text from `LNA` ports to top level layout.
2. Add dfpad.drawing - 41/0 along with TopMetal2.drawing on RF_INPUT/OUTPUT pins
3. Save as `FMD_QNC_04_160GHz_LNA.mod.gds` and upload to `/content/data/160GHz_LNA/design_data/gds`.

Set the project, top cell and gds file names.

In [4]:
%%shell
source /content/env
cat > $DATA_ROOT/project_env <<'EOF'
export PROJECT=160GHz_LNA
export LAYOUT_TOP=FMD_QNC_04_160GHz_LNA
export GDSFILE=design_data/gds/FMD_QNC_04_160GHz_LNA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $DATA_ROOT/project_env
source $DATA_ROOT/project_env
rm -f $DATA_ROOT/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT $DATA_ROOT/work
ls -l $DATA_ROOT/work

export PROJECT=160GHz_LNA
export LAYOUT_TOP=FMD_QNC_04_160GHz_LNA
export GDSFILE=design_data/gds/FMD_QNC_04_160GHz_LNA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
lrwxrwxrwx 1 root root 40 Aug  7 10:43 /content/data/work -> /content/data/160GHz_LNA/work/160GHz_LNA


Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [5]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$PROJECT",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "npn13G2*",
    "cmim*",
    "rfcmim*",
    "rsil*",
    "via_stack*",
    "GND",
    "sealring*",
    "*_stage",
    "LNA",
    "rhigh*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$DATA_ROOT/$PROJECT/$GDSFILE"
}
EOF

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export PROJECT=160GHz_LNA
export LAYOUT_TOP=FMD_QNC_04_160GHz_LNA
export GDSFILE=design_data/gds/FMD_QNC_04_160GHz_LNA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl


Write cvcrc file

In [6]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$PROJECT  <<EOF
GND power 0.0
VCC power 3.3
EOF

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export PROJECT=160GHz_LNA
export LAYOUT_TOP=FMD_QNC_04_160GHz_LNA
export GDSFILE=design_data/gds/FMD_QNC_04_160GHz_LNA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl


Create the klayout CDL file from the magic source netlist.

In [7]:
%%shell
source /content/env
sed -e '/^X[QMRCD]/s/.//' \
    -e "s/\.subckt $PROJECT/.subckt $LAYOUT_TOP/I" $SPICE_FILE > $KLAYOUT_CDL


export PROJECT=160GHz_LNA
export LAYOUT_TOP=FMD_QNC_04_160GHz_LNA
export GDSFILE=design_data/gds/FMD_QNC_04_160GHz_LNA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl


Results for 160GHz_LNA
*   layout has cells that have the same name as primitive devices. This causes netgen to segfault. <br>=> Flatglob all layout cells with primitive device names.
* Top layout does not have text. LNA level has text, but also has missing connectivity. <br>=> Copy text from LNA ports to top level.
* Top level met7 text on pads is not recognized in magic extraction. For magic, change tech file or switch to PADID. However, PADID not used in klayout.

# 2. 40_GHZ_LOW_NOISE_TIA

Checks will not work with GDS file in repo.
1. Copy TopMetal2.text from `TIA` to FDM_QNC_00_LN_TIA
2. Modify GND text position outside TopMetal2.pin
3. Modify text `VCC1`->`VCC3`, `VCC3`->`VCC1`
4. Save as `FMD_QNC_00_40_GHz_Low_Noise_TIA.mod.gds`
5. Upload to `/content/data/40_GHZ_LOW_NOISE_TIA/design_data/gds`

Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $DATA_ROOT/project_env <<'EOF'
export PROJECT=40_GHZ_LOW_NOISE_TIA
export LAYOUT_TOP=FDM_QNC_00_LN_TIA
export GDSFILE=design_data/gds/FMD_QNC_00_40_GHz_Low_Noise_TIA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $DATA_ROOT/project_env
source $DATA_ROOT/project_env
rm -f $DATA_ROOT/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT $DATA_ROOT/work
ls -l $DATA_ROOT/work


Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$PROJECT",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "npn13G2*",
    "cmim*",
    "rsil*",
    "rppd*",
    "GND",
    "sealring*",
    "TIA",
    "GND_PLANE",
    "*_STAGE",
    "VIA_STACK",
    "INPUT_PAD",
    "OUTPUT_PAD",
    "DC_PAD_DOWN",
    "DC_PAD_UP"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$DATA_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$PROJECT <<EOF
GND power 0.0
VCC* power 3.3
RFIN min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[QMRCD]/s/.//' \
    -e "s/\.subckt $PROJECT/.subckt $LAYOUT_TOP/I" $SPICE_FILE > $KLAYOUT_CDL

Results for 40_GHZ_LOW_NOISE_TIA
*   layout has cells that have the same name as primitive devices. This causes netgen to segfault. <br>=> Flatglob all layout cells with primitive device names.
* Top layout does not have text. `TIA` level has text, but also has missing connectivity. <br>=> Copy text from `TIA` ports to top level.
*  `VCC3` text does not overlap pin shape.<br>=> move text.
* `VCC1` an `VCC3` connections are reversed.![40GHz_klayout_lvs.png](https://github.com/d-m-bailey/ihp-mpw-be/blob/TO_Apr2025/TO_Apr2025/40_GHZ_LOW_NOISE_TIA/images/40GHz_klayout_lvs.png?raw=1)
* Top level met7 text on pads is not recognized in magic extraction. For magic, change tech file or switch to PADID. However, PADID is not used in klayout.
* Reported 1 CVC error:
```
    ! Short detected: 1257 to 0 Estimated current: 2.42mA
    /RX$ R rppd l=6u w=3u (r=520)
```

# 3. 6502-cpu


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=6502-cpu
export LAYOUT_TOP=cpu_top
export GDSFILE=design_data/gds/FMD_QNC_18_6502-cpu.2.gds
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for 6502-cpu (incomplete)
*   Missing top level power and ground.<br>=> Add top level pins for `iovss`, `iovdd`, `vdd`, and `vss`.
* Missing source netlist.
* `substrate` layer in `io` cells is not extracted as expected in magic.

# 4. 97_GHZ_LINEAR_TIA

Checks will not work with GDS file in repo.
1. Copy text from `TIA` ports to top level layout
2. Modify position of `GND` text outside TopMetal2.pin rects (5 points)
3. Remove `VB1`, `VB2` m2.pin/text (4 points) in STAGE_1, STAGE_4, BIAS_1, BIAS_2 cells
4. Save as `FMD_QNC_01_97_GHZ_LINEAR_TIA.mod.gds` and upload to `/content/data/97_GHZ_LINEAR_TIA/design_data/gds`.

Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $DATA_ROOT/project_env <<'EOF'
export PROJECT=97_GHZ_LINEAR_TIA
export LAYOUT_TOP=FMD_QNC_01_LIN_TIA
export GDSFILE=design_data/gds/FMD_QNC_01_97_GHZ_LINEAR_TIA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
# export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export SPICE_FILE=$WORK_ROOT/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $DATA_ROOT/project_env
source $DATA_ROOT/project_env
rm -f $DATA_ROOT/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT $DATA_ROOT/work
ls -l $DATA_ROOT/work

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$PROJECT",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "cmim*",
    "npn13G2*",
    "rhigh*",
    "rppd*",
    "rsil*",
    "sealring",
    "BIAS*",
    "DC_PAD_*",
    "FEEDBACK",
    "GND",
    "INPUT_*",
    "NSL",
    "OUTPUT_*",
    "STAGE*",
    "TIA",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$DATA_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$PROJECT <<EOF
GND power 0.0
VCC* power 3.3
RFIN min@0.0 max@3.3
EOF


Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[QMRCD]/s/.//' \
    -e "s/\.subckt $PROJECT/.subckt $LAYOUT_TOP/I" $SPICE_FILE > $KLAYOUT_CDL

Results for 97_GHZ_LINEAR_TIA
1. pcells with names matching primitive devices should be flattened in magic extraction.
2. Reported 1 CVC error which can be ignored
    ! Checking forward bias diode errors:
    /QX5 Q npn13g2 we=70n le=0.9u M=10 (r=1)
    B: RFIN

# 5. active_L_VCOs

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=active_L_VCOs
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for active_L_VCOs


# 6. ascon

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for ascon


# 7. bandgap_ref_cmos


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
** sch_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_2_full_bgr/bandgap_reference.sch
.subckt full_bandgap vdd VBG vss iout
*.PININFO vplus:B v-:B Vo1:B VBG:O
XM8 net1 v- vss vss sg13_lv_nmos l=10u w=150n ng=1 m=1
XM6 net1 net1 vdd vdd sg13_lv_pmos l=1u w=1u ng=1 m=1
XM7 net2 net1 vdd vdd sg13_lv_pmos l=1u w=1u ng=1 m=1
XM9 v- net2 vdd vdd sg13_lv_pmos l=4u w=200n ng=1 m=1
XM1 v- vplus vss vss sg13_lv_nmos l=5u w=7.14u ng=4 m=1
XM2 net3 net3 vss vss sg13_lv_nmos l=5u w=21u ng=8 m=1
XM3 v- Vo1 vdd vdd sg13_lv_pmos l=5u w=15u ng=8 m=1
XM4 vplus Vo1 vdd vdd sg13_lv_pmos l=5u w=15u ng=8 m=1
XM5 VBG Vo1 vdd vdd sg13_lv_pmos l=5u w=16u ng=8 m=1
*XC3 VBG vss cap_cmim w=72.965e-6 l=72.965e-6 m=1
XC3 vdd vss cap_cmim w=5e-6 l=5e-6 m=15
XR3 net3 vplus rppd w=0.5e-6 l=194.345e-6 m=1 b=0
*XR1 vss vplus rppd w=0.6e-6 l=194.345e-6 m=1 b=0
XR1_1 vss mid rppd w=03.0e-6 l=38.645e-6 m=1 b=0
XR1_2 mid vplus rppd w=0.5e-6 l=154.58e-6 m=1 b=0
*XR2 vss VBG rppd w=0.5e-6 l=192.395e-6 m=1 b=0
XR2 vss VBG rppd w=0.71e-6 l=270.515e-6 m=1 b=0
XC1 net2 vss cap_cmim w=18.195e-6 l=18.195e-6 m=1
x1 vdd iout vplus v- Vo1 vss two_stage_OTA
.ends

* expanding   symbol:  part_1_OTA/two_stage_OTA.sym # of pins=6
** sym_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_1_OTA/two_stage_OTA.sym
** sch_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_1_OTA/two_stage_OTA.sch
.subckt two_stage_OTA vdd iout vplus v- vout vss
*.PININFO v-:B vplus:B vss:B vdd:B iout:B vout:B
XM4 net3 net1 vss vss sg13_lv_nmos l=9.75u w=720n ng=1 m=1
XM3 net1 net1 vss vss sg13_lv_nmos l=9.75u w=720n ng=1 m=1
XM1 net1 v- net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM_dummy net1 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM2 net3 vplus net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM_dummy2 vdd vdd net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=4
XM_dummy3 net3 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM5 net2 iout vdd vdd sg13_lv_pmos l=1.95u w=5.3u ng=1 m=1
XM7 vout iout vdd vdd sg13_lv_pmos l=2.08u w=75u ng=8 m=1
XM6 vout net3 vss vss sg13_lv_nmos l=9.75u w=28.8u ng=4 m=1
XM9 iout iout vdd vdd sg13_lv_pmos l=2.08u w=75u ng=8 m=1
XC2 net3 vout cap_cmim w=22.295e-6 l=22.295e-6 m=1
.ends
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [ "" ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
vss power 0.0
vdd power 1.5
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[MRC]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for bandgap_ref_cmos
*   Pad text is on metal7 layer. This is not recognized on pads in magic. Either move the text to a region outside the pad, change the text layer to the PADID layer (41/0), or modify the magic extraction rules to recognize metal7 text on pads.
*   Klayout extraction rules generate n/ptap devices for layout taps annotated with "well"/"sub!" text. The schematic does not include these devices. Either add the devices to the schematic or remove the "well" and "sub!" text from the layout.
*   Ports do not match. Layout ports are vss, vdd, VBG, and iout. Netlist ports are GND, vdd, v+, v-, Vo1, and VBG. iout in the schematic is currently shorted to ground through an ammeter. Suggest removing v+, v-, Vo1 ports in schematic while adding iout port. Suggest changing GND from a global net to a port named vss in the schematic to match the layout.
* Layout has 3 dummy pmos (gate and source connected to vdd) not found in the schematic. Suggest adding them to the schematic. Suggest adding the following to `two_stage_OTA`
```
XM_dummy net1 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM_dummy2 vdd vdd net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=4
XM_dummy3 net3 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
```
* cap_cmim connected between VBG and vss in the schematic is connected between vdd and vss in the layout. The schematic size is 72.965 x 72.965 but the layout size is 5x5 m=15.
* R2 schematic size is 0.5/192.395 but layout is 0.71/38.645x7
* R1 schematic size is 0.6/194.345 but layout consists of resistors of different widths. 3.0/38.645 and 0.5/38.645x4. Currently netgen and klayout only reduce resistors with the same widths.

# 8. DC_to_130_GHz_TIA

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for DC_to_130_GHz_TIA


# 9. elemrv-n

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for elemrv-n


# 10. GPS_LNA

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for GPS_LNA


# 11. Greyhound

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for Greyhound


# 12. i2c-gpio-expander

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for i2c-gpio-expander


# 13. Mixer5GHz

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for Mixer5GHz


# 14. PA_180GHz

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for PA_180GHz


# 15. TTIHP0p2

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for TTIHP0p2


# 16. TTIHP25a

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for TTIHP25a


# 17. VCO_130nm_LSI

Not ready - do not run.


Set the project, top cell and gds file names.

In [ ]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<'EOF'
export PROJECT=ascon
export LAYOUT_TOP=
export GDSFILE=design_data/gds/
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work/$LAYOUT_TOP
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -f /content/data/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT
ln -s $WORK_ROOT /content/data/work
ls -l /content/data/work

Create the modified magic source netlist.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
.SUBCKT $LAYOUT_TOP
* no source
.ENDS
EOF

Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [
    "sg13g2_Corner",
    "sg13g2_Filler*",
    "sg13g2_IOPad*",
    "sg13g2_Clamp*",
    "sg13g2_RCClamp*",
    "sg13g2_DCNDiode",
    "sg13g2_DCPDiode",
    "sg13g2_GateDecode",
    "sg13g2_LevelDown",
    "sg13g2_SecondaryProtection",
    "sg13g2_LevelUp",
    "sg13g2_io_*",
    "sg13g2_*LevelUpInv",
    "TEXT*",
    "VIA*"
  ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}
EOF

Write cvcrc file

In [ ]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$LAYOUT_TOP <<EOF
*vss* power 0.0
iovdd* power 3.3
vdd power 1.5
clk input min@0.0 max@3.3
reset_n input min@0.0 max@3.3
EOF

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDMQR]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


Results for VCO_130nm_LSI


# Run BE checks

In [8]:
%%shell
cat /content/env
source /content/env
# run_be_checks includes magic/netgen soft connection and LVS, and CVC-RV
$LVS_ROOT/run_be_checks $WORK_ROOT/lvs_config.json

python3 $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/run_lvs.py --layout=$DATA_ROOT/$PROJECT/$GDSFILE \
  --netlist=$KLAYOUT_CDL \
  --run_dir=$WORK_ROOT/klayout \
  --topcell=$LAYOUT_TOP \
  --run_mode=deep \
  --spice_comments \
  --no_simplify \
  --combine_devices \
  --top_lvl_pins

LVS_BASENAME=$( echo $GDSFILE | sed -e 's,.*/,,' -e 's,\..*,,' )
$LVS_ROOT/check_klayout_ports $WORK_ROOT/klayout/$LVS_BASENAME.lvsdb > $WORK_ROOT/klayout/port_check.log
if [[ -e $WORK_ROOT/klayout/port_check.log ]]; then
  if [[ -s $WORK_ROOT/klayout/port_check.log ]]; then
    echo "* Klayout ports do not match *"
    echo "layout     source"
    awk '{printf "%-10s %s\n", $1, $2}' $WORK_ROOT/klayout/port_check.log
  else
    echo "Klayout ports match"
  fi
fi

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export PROJECT=160GHz_LNA
export LAYOUT_TOP=FMD_QNC_04_160GHz_LNA
export GDSFILE=design_data/gds/FMD_QNC_04_160GHz_LNA.mod.gds
export WORK_ROOT=$DATA_ROOT/$PROJECT/work/$PROJECT
export SPICE_FILE=$CHECK_ROOT/$MPW/$PROJECT/xschem/lvs/$PROJECT.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
Loading LVS environment from /content/data/160GHz_LNA/work/160G

# View verification logs

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')

# View soft verification report

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.report')

# View LVS report

In [ ]:
from google.colab import files

files.view('/content/data/work/lvs.report')

# View CVC report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.error')